# Hotspot Analysis for Car Accidents in New York City

In [6]:
import folium
from folium.plugins import HeatMap
from folium.plugins import MarkerCluster
from folium.plugins import TagFilterButton

import pandas as pd
from pathlib import Path

In [7]:
# Get File Path
dir_path = Path().resolve().parent
if (dir_path / Path('data_files/clean_data')).exists():
    data_path = dir_path / Path('data_files/clean_data')
else:
    (dir_path / Path('data_files/clean_data')).mkdir(parents=True, exist_ok=True)
    data_path = dir_path / Path('data_files/clean_data')

In [8]:
# Read in the data
model_data_path = Path(data_path / 'location_model_data.csv')
if model_data_path.exists():
    location_df = pd.read_csv(str(model_data_path), parse_dates=['CRASH_DATE'])
    location_df.head()
else:
    raise FileNotFoundError('Please run @02_accident_severity_model.ipynb first!')

,COLLISION_ID,CRASH_DATE,CRASH_TIME,LATITUDE,LONGITUDE,NUMBER_OF_PERSONS_INJURED,NUMBER_OF_PERSONS_KILLED,CONTRIBUTING_FACTOR_VEHICLE_1,CONTRIBUTING_FACTOR_VEHICLE_2,ACCIDENT_SEVERITY,...,BOROUGH_ENCODED,PERSON_TYPE_ENCODED,PERSON_SEX_ENCODED,BODILY_INJURY_ENCODED,EJECTION_ENCODED,YEAR,MONTH,DAY_OF_WEEK,IS_WEEKEND,SEASON
0,4273725,2020-01-14,22:00:00,40.860195,-73.828800,2,0,Driver Inattention/Distraction,Unspecified,Minor Injury,...,0,1,0,10,1,2020,1,1,0,1
1,4690423,2023-12-26,22:15:00,40.829422,-73.897575,1,0,Driver Inexperience,Unspecified,Minor Injury,...,0,2,1,9,1,2023,12,1,0,4
2,3844005,2018-02-11,13:00:00,40.878310,-73.870155,2,0,Turning Improperly,Unsafe Speed,Minor Injury,...,0,1,0,10,1,2018,2,6,1,1
3,4591181,2022-12-15,11:39:00,40.882630,-73.839516,1,0,Passing or Lane Usage Improper,Passing or Lane Usage Improper,Minor Injury,...,0,1,1,10,1,2022,12,3,0,4
4,3515925,2016-09-08,08:50:00,40.814266,-73.912964,1,0,Following Too Closely,Unspecified,Minor Injury,...,0,1,0,10,1,2016,9,3,0,3


In [9]:
location_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 189843 entries, 0 to 189842
Data columns (total 35 columns):
 #   Column                                 Non-Null Count   Dtype         
---  ------                                 --------------   -----         
 0   COLLISION_ID                           189843 non-null  int64         
 1   CRASH_DATE                             189843 non-null  datetime64[ns]
 2   CRASH_TIME                             189843 non-null  object        
 3   LATITUDE                               189843 non-null  float64       
 4   LONGITUDE                              189843 non-null  float64       
 5   NUMBER_OF_PERSONS_INJURED              189843 non-null  int64         
 6   NUMBER_OF_PERSONS_KILLED               189843 non-null  int64         
 7   CONTRIBUTING_FACTOR_VEHICLE_1          189843 non-null  object        
 8   CONTRIBUTING_FACTOR_VEHICLE_2          189843 non-null  object        
 9   ACCIDENT_SEVERITY                      189843 no

## Map of NYC Accidents 
This Map is very hard to read and understand. Cannot add Markers because of size of dataset (1M+) as well as the hardware specifications. We will need to be more specific in what insights we are looking for in this dataset.

In [10]:
boroughs = location_df['BOROUGH'].unique().tolist()
boroughs

['Bronx', 'Brooklyn', 'Manhattan', 'Queens', 'Staten Island']

In [11]:
location_df.shape

(189843, 35)

In [12]:
location_df['ACCIDENT_SEVERITY'].value_counts()

ACCIDENT_SEVERITY
Minor Injury    143283
Major Injury     45588
Fatal              959
No Injury           13
Name: count, dtype: int64

In [13]:
map_ny = folium.Map(location=[40.7128, -74.0060], zoom_start=12)


location_df = location_df.query("ACCIDENT_SEVERITY == 'Fatal' | ACCIDENT_SEVERITY == 'Major Injury'")
for x in location_df['BOROUGH'].unique():
    temp = location_df.query(f"BOROUGH == '{x}'")[['LATITUDE', 'LONGITUDE']].values.tolist()
    HeatMap(temp, radius=25, blur=15, max_zoom=13).add_to(map_ny)

# Tooltips:
marker_cluster = MarkerCluster().add_to(map_ny)
# Dictionary to Decode Label Encodings
season_dict = {1: 'Winter', 2: 'Spring', 3: 'Summer', 4: 'Fall'}
severity_dict = {'No Injury': 4, 'Minor Injury': 6, 'Major Injury': 8, 'Fatal': 10}

# Create the Markers on the Map
for idx, x in location_df.iterrows(): 
    collision_id = x.loc['COLLISION_ID']
    latitude = x.loc['LATITUDE']
    longitude = x.loc['LONGITUDE'] 

    text = (f"Accident Date: {str(x['CRASH_DATE']).split(' ')[0]} <br>"
             f"Accident Time: {x.loc['CRASH_TIME']} <br>"
             f"Accident Location: {latitude}, {longitude}<br>"
             f"Season: {season_dict[x.loc['SEASON'] ]}<br>"
            )
    text += "Info on People Involved: <br> -----------<br>"
    text += (f"Person Type: {x.loc['PERSON_TYPE']} <br>"
            f"Person Age: {x.loc['PERSON_AGE']} <br>"
              f"Injury Type: {x.loc['PERSON_INJURY']} <br>"
              f"Bodily Injury: {x.loc['BODILY_INJURY']} <br>"
              f"Emotional Status: {x.loc['EMOTIONAL_STATUS']} <br>" 
              f"---------------- <br>"
              )
    popup = folium.Popup(text, min_width=300, max_width=300)
    accident_severity = x.loc['ACCIDENT_SEVERITY']
    match accident_severity:
        case 'Fatal': 
            folium.Marker(
                location=[latitude, longitude],
                popup=popup,
                icon=folium.Icon(color='red'),
                tooltip=accident_severity, 
                tags=['Fatal']
            ).add_to(marker_cluster)
        case 'Major Injury': 
            folium.Marker(
                location=[latitude, longitude],
                popup=popup,
                icon=folium.Icon(color='blue'),
                tooltip=accident_severity,
                tags=['Major Injury']
            ).add_to(marker_cluster)
        case 'Minor Injury': 
            folium.Marker(
                location=[latitude, longitude],
                popup=popup,
                icon=folium.Icon(color='green'),
                tooltip=accident_severity,
                tags=['Minor Injury']
            ).add_to(marker_cluster)
        case 'No Injury': 
            folium.Marker(
                location=[latitude, longitude],
                popup=popup,
                icon=folium.Icon(color='purple'),
                tooltip=accident_severity,
                tags=['No Injury']
            ).add_to(marker_cluster)

TagFilterButton(list(severity_dict.keys())).add_to(map_ny)

In [14]:
map_ny.save(str(dir_path / Path('Visualizations') / "ny_accidents_fatal_heatmap.html"))